In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [10]:
load_dotenv()

True

In [22]:
# initialize Gemini model
model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",  # fast + free-tier friendly
    temperature=0.7
)

In [16]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

In [23]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from state
    question = state['question']

    # form a prompt
    prompt = f'Answer the following question {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [24]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [25]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

[{'type': 'text', 'text': 'The distance between the Earth and the Moon is not constant because the Moon follows an elliptical (oval-shaped) orbit.\n\nHere are the key measurements:\n\n*   **Average Distance:** Approximately **238,855 miles** (384,400 kilometers).\n*   **Perigee (Closest approach):** About **225,623 miles** (363,104 kilometers).\n*   **Apogee (Farthest point):** About **252,088 miles** (405,696 kilometers).\n\n**To put that distance into perspective:**\n*   **Light Speed:** It takes light about **1.3 seconds** to travel from the Moon to Earth.\n*   **Scale:** You could fit roughly **30 Earth-sized planets** in the gap between the Earth and the Moon.\n*   **Driving:** If you could drive a car at a constant 60 mph (100 km/h) toward the Moon, it would take you about **six months** to get there.', 'extras': {'signature': 'EvkECvYEAb4+9vtuCEZ4ee3ji6xJeva4nBxKCCE8npr7mvAOkUi4BttivAu+npbfngQoAWoVugDv2MShtrKdvnVafpaES+6+w6jnDNqoXYmR23Nzq2qMNo4LC4fV65jx6SVshKQfon0yQfvmPMOI7zpxmZ

In [27]:
model.invoke('How far is moon from the earth?').text

'The average distance from the Earth to the Moon is about **238,855 miles (384,400 kilometers).**\n\nHowever, because the Moon follows an elliptical (oval-shaped) orbit, this distance changes over time:\n\n*   **Perigee (closest point):** Approximately **225,623 miles** (363,104 km). At this point, the Moon appears larger and brighter in the sky (often called a "Supermoon").\n*   **Apogee (farthest point):** Approximately **252,088 miles** (405,696 km).\n\n### To put that distance into perspective:\n*   **The "30 Earths" Rule:** You could fit about 30 Earth-sized planets in the gap between the Earth and the Moon.\n*   **Travel by Car:** If you could drive to the Moon at 60 mph (96 km/h), it would take you about **six months** of non-stop driving to get there.\n*   **Speed of Light:** Light takes about **1.3 seconds** to travel from the Moon to Earth.\n*   **Apollo Missions:** It took the Apollo astronauts about **three days** to reach the Moon in a spacecraft.'